# Phase 2: Client-side Development

### Step 1: Tokenize the User Prompt Using TinyLlama’s Tokenizer

Load TinyLlama tokenizer

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
prompt = "Explain the benefits of homomorphic encryption."
tokens = tokenizer(prompt, return_tensors="pt")

/Users/rojo/Documents/repos/ZK-LLM-Turbo/split-inference-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Get input ID

In [2]:
input_ids = tokens["input_ids"]  # Shape: (1, sequence_length)
input_ids

tensor([[    1, 12027,  7420,   278, 23633,   310,  3632, 16898, 20956, 29889]])

### Step 2: Extract Embeddings Using the Input Embedding Layer


Load the TinyLlama model with only the embedding layer:

In [3]:
from transformers import AutoModelForCausalLM
import torch.nn as nn
import torch

model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
embedding_layer = model.model.embed_tokens  # nn.Embedding

Get the input embeddings

In [4]:
with torch.no_grad():
    embeddings = embedding_layer(input_ids)  # Shape: (1, seq_len, hidden_dim)

Convert to NumPy array for encryption:

In [5]:
embeddings_np = embeddings.squeeze(0).numpy()  # Shape: (seq_len, hidden_dim)
embeddings_np

array([[-1.0910034e-03,  1.9302368e-03, -1.6632080e-03, ...,
         1.9931793e-04, -6.5231323e-04, -4.9972534e-04],
       [ 6.4697266e-03,  3.7841797e-03, -1.9775391e-02, ...,
         4.5776367e-03,  7.9956055e-03,  5.4321289e-03],
       [-2.0141602e-02, -1.2329102e-02, -1.6235352e-02, ...,
         4.0588379e-03, -1.1352539e-02, -1.7456055e-02],
       ...,
       [ 6.5917969e-03, -1.7547607e-03,  2.2460938e-02, ...,
         2.8930664e-02, -6.8359375e-03, -7.0800781e-03],
       [-1.4160156e-02, -1.2664795e-03, -1.1230469e-02, ...,
        -2.7343750e-02, -2.8930664e-02,  1.7211914e-02],
       [-1.6860962e-03,  4.7922134e-05,  5.5847168e-03, ...,
        -4.6997070e-03,  1.1367798e-03, -5.0964355e-03]],
      shape=(10, 2048), dtype=float32)

### Step 3: Integrate CKKS Encryption (Python + SEAL)